## Classification supervisée des images

Un réseau de neurones est-il en mesure de détecter les armures des tissages ?

In [ ]:
import pandas as pd
import os
import shutil #copier fichier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import random

In [ ]:
from ultralytics import YOLO

In [ ]:
cd ../../data

### Création du dataset

### Téléchargement du modèle et visualisation de ses composantes

In [ ]:
#Version YOLOv8
model = YOLO('yolov8n-cls')

In [ ]:
#Version YOLOv8
model

## Premier entrainement avec dataset tel quel

### Partage des données train/test

In [ ]:
dataset_dir = 'dataset_technique/dataset'
train_dir = 'dataset_technique/data/train'
val_dir = 'dataset_technique/data/val'

In [ ]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
class_folders = [dir for dir in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, dir))]
#On récupère la listes des noms des dossiers/classes

In [ ]:
class_folders

### Entraînement des données

In [ ]:
for original_folder in class_folders:
    old_path  = os.path.join(dataset_dir, original_folder) #concaténation chemin et nom du dossier
    new_folder_name = original_folder.split('-')[0]
    new_path = os.path.join(dataset_dir, new_folder_name) #creation nouveau chemin
    
    os.rename(old_path, new_path)

In [ ]:
#Pour répartir les images d'entraînement et de validation dans des dossiers avec le bon nom
for class_folder in class_folders:
    class_path = os.path.join(dataset_dir, class_folder)
    images = [image for image in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, image))]
    
    train_img, val_img = train_test_split(images, test_size=0.2, random_state=42)
    
    train_subfolder_path = os.path.join(train_dir, class_folder)
    val_subfolder_path = os.path.join(val_dir, class_folder)
    
    os.makedirs(train_subfolder_path, exist_ok=True)
    os.makedirs(val_subfolder_path, exist_ok=True)
    
    for img in train_img:
        shutil.copy(os.path.join(class_path, img), train_subfolder_path)
        
    for img in val_img:
        shutil.copy(os.path.join(class_path, img), val_subfolder_path)

In [ ]:
###### Version pour YOLOv8
results = model.train(data='dataset_technique/data',
                      epochs=50,
                      batch=64,
                      dropout=0.1, 
                      seed = 42)
                     ,#device='mps') #Entrainement sur la puce Apple M2 (et M1) "Metal Performance Shaders"

#Bug. en utilisant MPS: error: 'mps.select' op failed to verify that all of {true_value, false_value, result} have same element type
    # note: see current operation: %17 = "mps.select"(%16, %15, %14) : (tensor<1xi1>, tensor<1xf16>, tensor<1xf32>) -> tensor<1xf16>
    #failed assertion 'expected a valid model URL' 
#Pas encore solutionné par Ultralytics : https://github.com/ultralytics/ultralytics/issues/7554 (issue du 13 janvier 2024).

In [ ]:
my_model = YOLO('runs/classify/train/weights/best.pt')
#Visualisation des résultats du modèle = résultat
result = my_model('dataset_technique/data/val/Mixedweave/BML031_IMG_3428.jpg', visualize=True, save=True)
#En jaune on voir les zones maximisées par neurones (pooling = plus on avance dans les couches moins on a de pixels)
#Intéressant pour comprendre les choix de classification d'un réseau (couleur, motif, background...)

#### Validation externe avec dataset de l'année dernière qui contient différentes techniques

In [ ]:
dataset_val_out = 'dataset_technique/test_ext'
images = [image for image in os.listdir(dataset_val_out) if image.endswith('.jpg')]
for elt in images :
    result = my_model(os.path.join(dataset_val_out, elt), visualize=True, save=True)

In [ ]:
#viabilité --> FID (f... inception distance), distance entre features d'images réelles et fausses. Images baveuses ou bruitées. 
#FID images réelles sur simages réelles et images réelles et images fausses.